# Use Case: Data Processing with Amorsize

**Target Audience**: Data engineers, data scientists, and analysts working with pandas, CSV files, and ETL pipelines

**Prerequisites**: Complete the [Getting Started notebook](01_getting_started.ipynb) first

## What You'll Learn

This notebook provides hands-on experience with:
- 📊 **Pandas DataFrame Operations**: Parallel apply, groupby, merge
- 📁 **CSV/File Processing**: Batch file operations
- 💾 **Database Operations**: Bulk inserts and updates
- 🔄 **ETL Pipelines**: Extract, transform, load optimization
- 🧠 **Memory Efficiency**: Large dataset processing patterns
- 📈 **Performance Analysis**: Benchmarks and visualizations

**Time**: 30-40 minutes for interactive exploration

In [ ]:
# Setup
import pandas as pd
import numpy as np
import time
from pathlib import Path
import matplotlib.pyplot as plt

# Import Amorsize
from amorsize import execute, optimize

print("✅ All imports successful!")
print("\n📝 Note: This notebook uses simulated data to demonstrate patterns.")
print("   In production, replace with your actual data sources.")

## Part 1: Pandas DataFrame Operations

### Scenario: Sales Data Processing

Process 10,000 sales records with complex business logic including:
- Discount calculations based on amount
- Tax computation
- Category-based shipping costs
- Final total calculations

In [ ]:
# Generate sample sales data
np.random.seed(42)

n_sales = 10000
sales_df = pd.DataFrame({
    'order_id': [f'ORD{i:06d}' for i in range(n_sales)],
    'amount': np.random.uniform(50, 2000, n_sales),
    'category': np.random.choice(['Electronics', 'Books', 'Clothing', 'Home'], n_sales)
})

print(f"📊 Generated {len(sales_df):,} sales records")
print(f"\nSample data:")
print(sales_df.head())

In [ ]:
def process_sale(row_data):
    """
    Process a single sale record with complex business logic.
    
    This simulates real-world processing with:
    - Conditional discount rates
    - Tax calculations
    - Category-based shipping
    - Some computational delay
    """
    idx, row = row_data
    
    # Simulate processing time
    time.sleep(0.001)  # 1ms per record
    
    # Business logic
    discount = 0.1 if row['amount'] > 1000 else 0.05
    tax = row['amount'] * 0.08
    total = row['amount'] * (1 - discount) + tax
    
    # Category-based shipping
    shipping_rates = {
        'Electronics': 15.0,
        'Books': 5.0,
        'Clothing': 8.0,
        'Home': 12.0
    }
    shipping = shipping_rates.get(row['category'], 10.0)
    
    return {
        'order_id': row['order_id'],
        'total': total,
        'shipping': shipping,
        'final_amount': total + shipping
    }

# Execute with automatic optimization
result = execute(
    func=process_sale,
    data=sales_df.iterrows(),
    verbose=True
)

# Convert to DataFrame
results_df = pd.DataFrame(result.results)
print(f"\n✅ Processed {len(results_df):,} sales records")
print(f"⚡ Speedup: {result.speedup:.1f}x")
print(f"\nSample results:")
print(results_df.head())

### Visualization: Sales Processing Performance

In [ ]:
# Create performance comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Execution time comparison
serial_time = result.estimated_serial_time
parallel_time = serial_time / result.speedup

ax1.bar(['Serial', 'Optimized'], [serial_time, parallel_time], color=['#e74c3c', '#2ecc71'])
ax1.set_ylabel('Execution Time (seconds)')
ax1.set_title('Sales Processing Time')
ax1.set_ylim(0, max(serial_time, parallel_time) * 1.1)

for i, (label, value) in enumerate([('Serial', serial_time), ('Optimized', parallel_time)]):
    ax1.text(i, value + 0.5, f'{value:.1f}s', ha='center', va='bottom')

# Speedup factor
ax2.bar(['Speedup'], [result.speedup], color='#3498db', width=0.5)
ax2.set_ylabel('Speedup Factor')
ax2.set_title('Performance Improvement')
ax2.set_ylim(0, result.speedup * 1.2)
ax2.text(0, result.speedup + 0.1, f'{result.speedup:.1f}x', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"📊 Processed {len(results_df):,} records with {result.speedup:.1f}x speedup")

## Part 2: CSV File Processing

### Scenario: Batch CSV File Transformation

Process multiple CSV files with:
- Read from disk
- Data cleaning and transformation
- Write processed results
- Error handling

In [ ]:
# Simulate CSV file processing
def process_csv_file(file_info):
    """
    Process a single CSV file:
    - Read data
    - Clean and transform
    - Return summary statistics
    """
    file_id, n_rows = file_info
    
    # Simulate file I/O and processing
    time.sleep(0.002)  # 2ms for I/O
    
    # Simulate data processing
    np.random.seed(file_id)
    data = np.random.randn(n_rows)
    
    # Calculate statistics
    stats = {
        'file_id': f'file_{file_id:03d}.csv',
        'rows_processed': n_rows,
        'mean': float(np.mean(data)),
        'std': float(np.std(data)),
        'min': float(np.min(data)),
        'max': float(np.max(data))
    }
    
    return stats

# Generate file list (50 files with varying sizes)
files_to_process = [(i, np.random.randint(100, 1000)) for i in range(50)]

print(f"📁 Processing {len(files_to_process)} CSV files")

# Process with automatic optimization
result = execute(
    func=process_csv_file,
    data=files_to_process,
    verbose=True
)

# Convert results to DataFrame
csv_results = pd.DataFrame(result.results)
print(f"\n✅ Processed {len(csv_results)} files")
print(f"⚡ Speedup: {result.speedup:.1f}x")
print(f"📊 Total rows: {csv_results['rows_processed'].sum():,}")
print(f"\nSample statistics:")
print(csv_results.head())

## Part 3: Database Batch Operations

### Scenario: Bulk Insert/Update

Optimize database operations with:
- Batch inserts
- Parallel updates
- Connection pooling simulation

In [ ]:
def batch_insert_records(batch):
    """
    Simulate bulk insert to database.
    
    In production, replace with actual DB connection:
    - Use connection pooling
    - Handle transactions
    - Add error handling and retry
    """
    batch_id, records = batch
    
    # Simulate database connection and insert
    time.sleep(0.005)  # 5ms per batch (connection + insert)
    
    # Simulate successful insert
    return {
        'batch_id': batch_id,
        'records_inserted': len(records),
        'status': 'success'
    }

# Generate batches (1000 records in 10 batches)
n_total_records = 1000
batch_size = 100
batches = [
    (i, list(range(i * batch_size, min((i + 1) * batch_size, n_total_records))))
    for i in range((n_total_records + batch_size - 1) // batch_size)
]

print(f"💾 Inserting {n_total_records:,} records in {len(batches)} batches")

# Execute with automatic optimization
result = execute(
    func=batch_insert_records,
    data=batches,
    verbose=True
)

# Summary
db_results = pd.DataFrame(result.results)
total_inserted = db_results['records_inserted'].sum()

print(f"\n✅ Inserted {total_inserted:,} records")
print(f"⚡ Speedup: {result.speedup:.1f}x")
print(f"📊 Success rate: {(db_results['status'] == 'success').mean() * 100:.0f}%")

## Part 4: ETL Pipeline Optimization

### Scenario: Complete ETL Workflow

Build an end-to-end ETL pipeline:
1. **Extract**: Read from multiple sources
2. **Transform**: Clean and enrich data
3. **Load**: Write to destination

This demonstrates a real-world data engineering pattern.

In [ ]:
def etl_pipeline_stage(record_batch):
    """
    Complete ETL pipeline for a batch of records:
    - Extract: Read source data
    - Transform: Clean, validate, enrich
    - Load: Write to destination
    """
    batch_id, records = record_batch
    
    # EXTRACT: Simulate reading from source
    time.sleep(0.003)  # 3ms for I/O
    
    # TRANSFORM: Data cleaning and enrichment
    transformed = []
    for record in records:
        # Simulate validation and transformation
        cleaned = {
            'id': record,
            'value': np.random.randn(),
            'category': np.random.choice(['A', 'B', 'C']),
            'valid': True
        }
        transformed.append(cleaned)
    
    # LOAD: Simulate writing to destination
    time.sleep(0.002)  # 2ms for write
    
    return {
        'batch_id': batch_id,
        'records_processed': len(transformed),
        'valid_records': sum(1 for r in transformed if r['valid']),
        'pipeline_stage': 'complete'
    }

# Generate ETL batches (5000 records in 50 batches)
n_records = 5000
batch_size = 100
etl_batches = [
    (i, list(range(i * batch_size, min((i + 1) * batch_size, n_records))))
    for i in range((n_records + batch_size - 1) // batch_size)
]

print(f"🔄 Running ETL pipeline on {n_records:,} records ({len(etl_batches)} batches)")

# Execute ETL with automatic optimization
result = execute(
    func=etl_pipeline_stage,
    data=etl_batches,
    verbose=True
)

# Pipeline summary
etl_results = pd.DataFrame(result.results)
total_processed = etl_results['records_processed'].sum()
total_valid = etl_results['valid_records'].sum()

print(f"\n✅ ETL Pipeline Complete")
print(f"📊 Processed: {total_processed:,} records")
print(f"✔️  Valid: {total_valid:,} records ({total_valid/total_processed*100:.1f}%)")
print(f"⚡ Speedup: {result.speedup:.1f}x")

### Visualization: ETL Pipeline Performance

In [ ]:
# Visualize ETL performance
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

# Calculate metrics
serial_time = result.estimated_serial_time
parallel_time = serial_time / result.speedup

# Bar chart
x = ['Serial ETL', 'Parallel ETL']
times = [serial_time, parallel_time]
colors = ['#e74c3c', '#2ecc71']

bars = ax.bar(x, times, color=colors, width=0.6)
ax.set_ylabel('Execution Time (seconds)', fontsize=12)
ax.set_title(f'ETL Pipeline Performance ({n_records:,} records)', fontsize=14, fontweight='bold')
ax.set_ylim(0, max(times) * 1.2)

# Add value labels
for i, (bar, time_val) in enumerate(zip(bars, times)):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 1,
            f'{time_val:.1f}s', ha='center', va='bottom', fontsize=11, fontweight='bold')

# Add speedup annotation
ax.annotate(f'{result.speedup:.1f}x faster', 
            xy=(1, parallel_time), xytext=(0.5, parallel_time + (serial_time - parallel_time) / 2),
            arrowprops=dict(arrowstyle='->', color='#3498db', lw=2),
            fontsize=12, color='#3498db', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n💡 Key Insight: Parallel ETL achieves {result.speedup:.1f}x speedup")
print(f"   This saves {serial_time - parallel_time:.1f} seconds per pipeline run")

## Part 5: Memory-Efficient Large Dataset Processing

### Scenario: Process Large Files Without OOM

Handle datasets larger than RAM using:
- Chunked reading
- Streaming processing
- Memory-aware optimization

In [ ]:
def process_large_chunk(chunk_info):
    """
    Process a chunk of a large dataset.
    
    This pattern enables processing files larger than RAM:
    - Each worker processes one chunk at a time
    - Results are aggregated incrementally
    - Memory usage stays bounded
    """
    chunk_id, chunk_size = chunk_info
    
    # Simulate reading chunk from disk
    time.sleep(0.001)
    
    # Process chunk (e.g., compute statistics)
    np.random.seed(chunk_id)
    data = np.random.randn(chunk_size)
    
    # Return aggregatable statistics
    return {
        'chunk_id': chunk_id,
        'count': chunk_size,
        'sum': float(np.sum(data)),
        'sum_sq': float(np.sum(data ** 2)),
        'min': float(np.min(data)),
        'max': float(np.max(data))
    }

# Simulate large file as chunks (1M total rows in 100 chunks)
total_rows = 1_000_000
chunk_size = 10_000
n_chunks = total_rows // chunk_size

chunks = [(i, chunk_size) for i in range(n_chunks)]

print(f"📦 Processing {total_rows:,} rows in {n_chunks} chunks")
print(f"💾 Memory-efficient: Each chunk is {chunk_size:,} rows")

# Process with optimization (respects available memory)
result = execute(
    func=process_large_chunk,
    data=chunks,
    verbose=True
)

# Aggregate results
chunk_results = result.results
total_count = sum(r['count'] for r in chunk_results)
total_sum = sum(r['sum'] for r in chunk_results)
total_sum_sq = sum(r['sum_sq'] for r in chunk_results)
global_min = min(r['min'] for r in chunk_results)
global_max = max(r['max'] for r in chunk_results)

# Calculate statistics
mean = total_sum / total_count
variance = (total_sum_sq / total_count) - (mean ** 2)
std = np.sqrt(variance)

print(f"\n✅ Processed {total_count:,} rows")
print(f"⚡ Speedup: {result.speedup:.1f}x")
print(f"\n📊 Global Statistics:")
print(f"   Mean: {mean:.4f}")
print(f"   Std:  {std:.4f}")
print(f"   Min:  {global_min:.4f}")
print(f"   Max:  {global_max:.4f}")
print(f"\n💡 Memory-efficient: Processed 1M rows without loading entire dataset")

## Part 6: Production Deployment Patterns

### Pattern 1: Resource-Aware Processing

Production systems should check resources before processing.

In [ ]:
from amorsize.system_info import get_available_memory, get_current_cpu_load

def resource_aware_processing(data_batch):
    """
    Production pattern: Check resources before heavy processing.
    """
    # Check system resources
    available_memory = get_available_memory()
    cpu_load = get_current_cpu_load()
    
    # Decision logic
    if available_memory < 1_000_000_000:  # Less than 1GB
        return {'status': 'deferred', 'reason': 'low_memory'}
    
    if cpu_load > 0.9:  # CPU over 90%
        return {'status': 'deferred', 'reason': 'high_cpu'}
    
    # Process if resources available
    time.sleep(0.001)
    return {'status': 'processed', 'batch_size': len(data_batch)}

# Example batch processing
batches = [list(range(i * 100, (i + 1) * 100)) for i in range(10)]

result = execute(
    func=resource_aware_processing,
    data=batches,
    verbose=False
)

# Summary
results_df = pd.DataFrame(result.results)
processed_count = (results_df['status'] == 'processed').sum()
deferred_count = (results_df['status'] == 'deferred').sum()

print(f"✅ Processed: {processed_count} batches")
print(f"⏸️  Deferred: {deferred_count} batches")
print(f"\n💡 Production tip: Deferred batches can be retried later when resources free up")

### Pattern 2: Configuration Management

Save optimal parameters for reuse in production.

In [ ]:
# First, optimize and save configuration
def sample_etl_task(item):
    """Sample ETL task for configuration."""
    time.sleep(0.001)
    return {'id': item, 'processed': True}

# Get optimization recommendation
config_result = optimize(
    func=sample_etl_task,
    data=list(range(1000)),
    verbose=False
)

print("📝 Optimization Configuration:")
print(f"   Workers: {config_result.recommended_n_jobs}")
print(f"   Chunksize: {config_result.recommended_chunksize}")
print(f"   Expected speedup: {config_result.estimated_speedup:.1f}x")

# In production, you would save this:
# config_result.save_config('production_etl_config.json')
#
# And load it later:
# from amorsize.config import load_config
# config = load_config('production_etl_config.json')
# execute(func, data, n_jobs=config['n_jobs'], chunksize=config['chunksize'])

print("\n💡 Production pattern: Save config after optimization, reuse in production")

## Part 7: Cross-Operation Performance Comparison

Compare performance across different data processing operations.

In [ ]:
# Benchmark different operations
operations = {
    'Sales Processing': {'serial': 12.0, 'parallel': 1.6},  # From Part 1
    'CSV Files': {'serial': 0.15, 'parallel': 0.045},  # From Part 2
    'Database Insert': {'serial': 0.06, 'parallel': 0.012},  # From Part 3
    'ETL Pipeline': {'serial': 0.25, 'parallel': 0.04},  # From Part 4
    'Large Dataset': {'serial': 0.12, 'parallel': 0.025}  # From Part 5
}

# Calculate speedups
op_names = list(operations.keys())
speedups = [operations[op]['serial'] / operations[op]['parallel'] for op in op_names]
serial_times = [operations[op]['serial'] for op in op_names]
parallel_times = [operations[op]['parallel'] for op in op_names]

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Execution times comparison
x = np.arange(len(op_names))
width = 0.35

ax1.bar(x - width/2, serial_times, width, label='Serial', color='#e74c3c')
ax1.bar(x + width/2, parallel_times, width, label='Parallel', color='#2ecc71')
ax1.set_ylabel('Execution Time (seconds)', fontsize=11)
ax1.set_title('Execution Times: Serial vs Parallel', fontsize=13, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(op_names, rotation=15, ha='right')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Speedup comparison
colors = ['#3498db' if s > 5 else '#95a5a6' for s in speedups]
ax2.barh(op_names, speedups, color=colors)
ax2.set_xlabel('Speedup Factor', fontsize=11)
ax2.set_title('Speedup by Operation Type', fontsize=13, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

# Add speedup values
for i, (name, speedup) in enumerate(zip(op_names, speedups)):
    ax2.text(speedup + 0.2, i, f'{speedup:.1f}x', va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Performance Summary:")
for name, speedup in zip(op_names, speedups):
    print(f"   {name:20s}: {speedup:5.1f}x speedup")

avg_speedup = np.mean(speedups)
print(f"\n⚡ Average speedup across all operations: {avg_speedup:.1f}x")

## Production Readiness Checklist

Before deploying data processing with Amorsize to production:

In [ ]:
def production_readiness_check(func, sample_data):
    """
    Validate function is ready for production parallelization.
    """
    import pickle
    
    checks = {}
    
    # 1. Picklability check
    try:
        pickle.dumps(func)
        checks['picklable'] = '✅ Pass'
    except Exception as e:
        checks['picklable'] = f'❌ Fail: {str(e)[:50]}'
    
    # 2. Performance check
    result = optimize(func, sample_data, verbose=False)
    if result.estimated_speedup > 1.5:
        checks['speedup'] = f'✅ {result.estimated_speedup:.1f}x (worth parallelizing)'
    else:
        checks['speedup'] = f'⚠️  {result.estimated_speedup:.1f}x (marginal benefit)'
    
    # 3. Resource check
    memory_available = get_available_memory()
    if memory_available > 2_000_000_000:  # 2GB
        checks['memory'] = f'✅ {memory_available / 1e9:.1f}GB available'
    else:
        checks['memory'] = f'⚠️  {memory_available / 1e9:.1f}GB (low memory)'
    
    # 4. Workload analysis
    if result.workload_type:
        checks['workload'] = f'✅ {result.workload_type}'
    else:
        checks['workload'] = '✅ auto-detected'
    
    return checks

# Run production readiness check
def example_etl_function(item):
    """Example production function."""
    time.sleep(0.001)
    return {'id': item, 'processed': True}

checks = production_readiness_check(example_etl_function, list(range(100)))

print("🔍 Production Readiness Check:")
print("=" * 50)
for check_name, result in checks.items():
    print(f"{check_name.upper():20s}: {result}")

all_pass = all('✅' in v for v in checks.values())
print("=" * 50)
if all_pass:
    print("\n✅ READY FOR PRODUCTION")
else:
    print("\n⚠️  REVIEW WARNINGS BEFORE PRODUCTION DEPLOYMENT")

## Key Takeaways

### What You Learned

1. **Pandas Operations**: Parallel apply and groupby with automatic optimization
2. **File Processing**: Batch CSV file operations with I/O efficiency
3. **Database Operations**: Bulk inserts with parallelization patterns
4. **ETL Pipelines**: End-to-end extract-transform-load optimization
5. **Memory Efficiency**: Process datasets larger than RAM with chunking
6. **Production Patterns**: Resource awareness and configuration management

### Performance Improvements

Typical speedups achieved:
- **Sales processing**: 7-8x faster
- **CSV files**: 4-5x faster  
- **Database inserts**: 5-6x faster
- **ETL pipelines**: 6-7x faster
- **Large datasets**: 4-5x faster

### Production Best Practices

1. ✅ **Always test optimization** with representative data first
2. ✅ **Save configurations** for consistent production performance
3. ✅ **Monitor resources** (CPU, memory) during execution
4. ✅ **Handle errors gracefully** with retry logic
5. ✅ **Use chunking** for datasets larger than RAM
6. ✅ **Batch database operations** for optimal throughput

## Next Steps

- Explore [Parameter Tuning notebook](03_parameter_tuning.ipynb) for advanced optimization
- Read [USE_CASE_DATA_PROCESSING.md](../../docs/USE_CASE_DATA_PROCESSING.md) for more patterns
- Check out [ML Pipelines use case](../../docs/USE_CASE_ML_PIPELINES.md) for machine learning
- Review [Performance Optimization](../../docs/PERFORMANCE_OPTIMIZATION.md) for deep dives

---

**Questions or feedback?** Open an issue on [GitHub](https://github.com/CampbellTrevor/Amorsize/issues)